<a href="https://colab.research.google.com/github/debabratabepari/Data-Science-Practice/blob/main/amazon_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
 import pandas as pd

In [138]:
df = pd.read_csv("amazon_cells_labelled.csv")

In [139]:
df.head()

,Title,Label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [140]:

df.shape

(1000, 2)

In [141]:
df.isnull().sum()

Title    0
Label    0
dtype: int64

In [142]:
x = df.drop("Label",axis=1)

In [143]:
y = df["Label"]

In [144]:
x.head()

,Title
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In [145]:
y.head()

0    0
1    1
2    1
3    0
4    1
Name: Label, dtype: int64

In [146]:
import tensorflow as tf


In [147]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [148]:

voc_size= 5000

In [149]:
review = x.copy()

In [150]:
review.head()

,Title
0,So there is no way for me to plug it in here i...
1,"Good case, Excellent value."
2,Great for the jawbone.
3,Tied to charger for conversations lasting more...
4,The mic is great.


In [151]:
# type(review)

In [152]:
import nltk
import re
from nltk.corpus import stopwords

In [153]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [154]:
review['Title'][0]

'So there is no way for me to plug it in here in the US unless I go by a converter.'

In [155]:
# stopwords.words('english')

### Preprocessing Part

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

corpus = []
for i in range(0,len(review)):
  temp = re.sub("[^a-zA-Z]",' ',review['Title'][i])
  temp = temp.lower()
  temp = temp.split()

  temp = [ps.stem(word) for word in temp if not word in stopwords.words('english')]
  temp = ' '.join(temp)
  corpus.append(temp)
corpus

In [157]:

onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4088, 733, 3593, 3162, 2846, 3947],
 [518, 1118, 560, 3559],
 [2451, 518],
 [3423, 2122, 2256, 3971, 223, 307, 2343],
 [3461, 2451],
 [1944, 733, 1454, 3356, 1201, 1454, 3867, 4178],
 [4264, 600, 4264, 1711, 1561, 4963, 1130, 1981, 3648, 3648],
 [3571, 1231, 999],
 [1670, 854, 2955, 1938],
 [2955, 1938, 2780],
 [2991, 2837, 2451],
 [3266, 2846, 3846, 853, 239, 853],
 [1346, 4433, 3419, 171, 4163, 946, 1593, 1994, 84, 2991, 4972],
 [518, 2837, 3988],
 [2075, 270, 4359, 2775, 4123],
 [2539, 2122, 3648, 4686, 2474, 261],
 [1043, 4286, 828],
 [2930, 518],
 [3293, 2451],
 [3855, 4564, 4088, 2884, 3527, 3509, 4522, 2047, 3971],
 [397, 1708, 1993, 3526, 2009, 4974, 1454, 1342],
 [3821, 541, 4319, 4321, 2986, 2606],
 [3806, 4395],
 [1267, 4873, 4147, 853, 1346, 1832, 3372, 2067, 1479, 4435],
 [3821, 3460, 2343, 853],
 [2451, 311, 467, 261, 4302],
 [4279, 261, 4343, 854, 4330, 595, 261],
 [2366, 3402, 3358, 4988],
 [392, 93, 4988, 3507, 502, 2343, 3507, 261],
 [1450, 4435],
 [2734, 3912, 261,

### Embedding Representation


In [158]:
# sen_len= []
# for words in corpus:
#   sen_len.append(len(words))
# max(sen_len)


In [159]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,maxlen= sent_length, padding='post')


In [160]:
len(embedded_docs),y.shape

(1000, (1000,))

In [161]:
embedded_docs[413]

array([ 818, 2634, 4429,  853, 3231, 2568,  280,  653, 3846,  541, 3971,
       2870, 4146, 3971, 2967, 2894,    0,    0,    0,    0], dtype=int32)

In [162]:
from tensorflow.keras.layers import Dropout
embedding_vector_features= 50
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length= sent_length))
# model.add(Dropout(0.3))
model.add(LSTM(150, dropout= 0.2, recurrent_dropout= 0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation = "sigmoid"))
model.compile(loss= 'binary_crossentropy', optimizer= 'adam', metrics= ['accuracy'])

In [163]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 50)            250000    
                                                                 
 lstm_3 (LSTM)               (None, 150)               120600    
                                                                 
 dense_4 (Dense)             (None, 64)                9664      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 380329 (1.45 MB)
Trainable params: 380329 (1.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [164]:
import numpy as np
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [165]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.2, random_state=42)

In [166]:
x_test

array([[3146, 1077, 3743, ...,    0,    0,    0],
       [1847, 2519, 4371, ...,    0,    0,    0],
       [2047, 2539, 2122, ...,    0,    0,    0],
       ...,
       [ 518, 4161, 4883, ...,    0,    0,    0],
       [4115, 4491, 2375, ...,    0,    0,    0],
       [2606,  261, 3689, ...,    0,    0,    0]], dtype=int32)

### Model Training

In [167]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)


Epoch 1/10
13/13 [==============================] - 6s 145ms/step - loss: 0.6949 - accuracy: 0.4837 - val_loss: 0.6931 - val_accuracy: 0.5350
Epoch 2/10
13/13 [==============================] - 2s 132ms/step - loss: 0.6933 - accuracy: 0.5038 - val_loss: 0.6933 - val_accuracy: 0.4650
Epoch 3/10
13/13 [==============================] - 1s 79ms/step - loss: 0.6940 - accuracy: 0.4787 - val_loss: 0.6932 - val_accuracy: 0.4650
Epoch 4/10
13/13 [==============================] - 1s 78ms/step - loss: 0.6932 - accuracy: 0.5138 - val_loss: 0.6943 - val_accuracy: 0.4650
Epoch 5/10
13/13 [==============================] - 1s 78ms/step - loss: 0.6852 - accuracy: 0.5175 - val_loss: 0.6379 - val_accuracy: 0.6550
Epoch 6/10
13/13 [==============================] - 1s 78ms/step - loss: 0.4690 - accuracy: 0.8087 - val_loss: 0.5642 - val_accuracy: 0.7200
Epoch 7/10
13/13 [==============================] - 1s 77ms/step - loss: 0.2822 - accuracy: 0.9000 - val_loss: 0.6386 - val_accuracy: 0.7550
Epoch 8/10


In [168]:
y_pred = model.predict(x_test)
# y_pred

7/7 [==============================] - 1s 17ms/step


In [169]:
# import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Compute ROC curve and ROC area for each class
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

# # Plot ROC curve
# plt.figure()
# lw = 2
# plt.plot(fpr, tpr, color='darkorange',lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) Curve')
# plt.legend(loc="lower right")
# plt.show()

# Create ROC curve plotly figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC curve (area = %0.2f)' % roc_auc))
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'), name='Random Guess'))

# Update layout
fig.update_layout(
    title='Receiver Operating Characteristic (ROC)',
    xaxis=dict(title='False Positive Rate'),
    yaxis=dict(title='True Positive Rate'),
    legend=dict(x=0.05, y=0.95),
    margin=dict(l=20, r=20, t=60, b=20),
    hovermode='closest'
)

# Show plot
fig.show()

In [171]:
y_pred = np.where(y_pred>0.87, 1, 0)

In [172]:
from sklearn.metrics import confusion_matrix

In [173]:
confusion_matrix(y_test, y_pred)

array([[81, 12],
       [30, 77]])

In [174]:
from sklearn.metrics import accuracy_score

In [175]:
accuracy_score(y_test, y_pred)

0.79

In [176]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.87      0.79        93
           1       0.87      0.72      0.79       107

    accuracy                           0.79       200
   macro avg       0.80      0.80      0.79       200
weighted avg       0.80      0.79      0.79       200



###Model Checking

In [177]:
# t=['i like this product', 'i will never buy this again']
# def docs(t):
  # doc = pd.DataFrame(t)
  # doc.columns = ['Title']
#   return(doc)
# docs(text)


In [178]:
def data_prep(text):
  doc = pd.DataFrame(text)
  doc.columns = ['Title']
  corpus = []
  for i in range(0,len(doc)):
    temp = re.sub("[^a-zA-Z]",' ',doc['Title'][i])
    temp = temp.lower()
    temp = temp.split()

    temp = [ps.stem(word) for word in temp if not word in stopwords.words('english')]
    temp = ' '.join(temp)
    corpus.append(temp)
  onehot_repr = [one_hot(words,voc_size) for words in corpus]
  embedded_docs = pad_sequences(onehot_repr,maxlen= 20, padding='post')
  return embedded_docs;
# data_prep(review)

In [179]:
f=["Use less product and performance . Appearance bad , smoothness rough , battery life only 20 to 25 minutes value for money not Just i perched this laptop 29 March so slow and too much hang.",
   "This laptop is is very good performance prize is very high b,t sound quality is very low and heting issue",
   "Fingerprint lock option is not working in laptop"]

In [180]:
# t=['i like this product', 'i will never buy this again']
p=data_prep(f)
# p

In [181]:
pred= model.predict(p)
pred = np.where(pred>0.86,1,0)
pred


1/1 [==============================] - 0s 25ms/step


array([[0],
       [1],
       [0]])

In [43]:
# from sklearn.pipeline import Pipeline
# my_pipeline = Pipeline([
#     ('data', data_prep),
#     ('model', Sequential())
# ])